In [1]:
import os
import time
from selenium.webdriver import Firefox, FirefoxService
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
import pandas as pd

In [2]:
indexes = {'WORLD': '990100',
           'ACWI':  '892400',
           'WORLD VALUE': '105868',
           'ACWI VALUE': '106039',
           'WORLD GROWTH': '105867',
           'ACWI GROWTH': '106038'}

In [3]:
indexes = {
    'WORLD SMALL CAP': '106230',
    'WORLD SMALL VALUE': '655131',
    'WORLD SMALL GROWTH': '655236',
    'ACWI SMALL CAP': '655045',
    'ACWI SMALL VALUE': '655150',
    'ACWI SMALL GROWTH': '655255',
}

In [4]:
indexes = {
    'EM (EMERGING MARKETS)': '891800',
    'EM (EMERGING MARKETS) VALUE': '106063',
    'EM (EMERGING MARKETS) GROWTH': '106062',
    'EM (EMERGING MARKETS) SMALL CAP': '655061',
    'EM (EMERGING MARKETS) SMALL VALUE': '655166',
    'EM (EMERGING MARKETS) SMALL GROWTH': '655271',
}

In [5]:
indexes = {
    'ACWI MID CAP': '652574',
    'ACWI MID VALUE': '652679',
    'ACWI MID GROWTH': '652784',
    'WORLD MID CAP': '652555',
    'WORLD MID VALUE': '652660',
    'WORLD MID GROWTH': '652758',
    'EM (EMERGING MARKETS) MID CAP': '652590',
    'EM (EMERGING MARKETS) MID VALUE': '652695',
    'EM (EMERGING MARKETS) MID GROWTH': '652800',
}

In [6]:
price_levels = {'Gross': 'GRTR',
                'Net': 'NETR'}

In [7]:
[(key, value) for (key, value) in indexes.items()]

[('ACWI MID CAP', '652574'),
 ('ACWI MID VALUE', '652679'),
 ('ACWI MID GROWTH', '652784'),
 ('WORLD MID CAP', '652555'),
 ('WORLD MID VALUE', '652660'),
 ('WORLD MID GROWTH', '652758'),
 ('EM (EMERGING MARKETS) MID CAP', '652590'),
 ('EM (EMERGING MARKETS) MID VALUE', '652695'),
 ('EM (EMERGING MARKETS) MID GROWTH', '652800')]

In [8]:
service = FirefoxService(executable_path="C:\\Users\\leowy\\anaconda3\\envs\\market-returns\\Scripts\\geckodriver.exe")

C:\Users\leowy\AppData\Local\Temp\ipykernel_11492\4012756472.py:1: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  service = FirefoxService(executable_path="C:\\Users\\leowy\\anaconda3\\envs\\market-returns\\Scripts\\geckodriver.exe")


In [9]:
driver = Firefox(service=service)

In [10]:
driver.implicitly_wait(2.)

In [11]:
for index, code in indexes.items():
    for index_level, price_level in price_levels.items():
        driver.get(f'https://app2.msci.com/products/index-data-search/regional_chart.jsp?currency=USD&priceLevel={price_level}&indexId={code}&indexName={index}')
        WebDriverWait(driver, 60).until(EC.invisibility_of_element(('id', 'loading-image-graph')))
        time.sleep(1)
        if not os.path.exists(f'c:\\Users\\leowy\\Downloads\\MSCI {index} USD {index_level} Monthly.xls'):
            term_field = driver.find_element('id', 'updateTermRegion')
            term_field_select = Select(term_field)
            term_field_select.select_by_value('Full')
            driver.find_element('xpath', '/html/body/div[1]/div/section/div/div[4]/div[4]/div[2]/form/div/div/div[8]/div/div/button').click()
            WebDriverWait(driver, 60).until(EC.invisibility_of_element(('id', 'loading-image-graph')))
            time.sleep(1)
            driver.find_element('xpath', '/html/body/div[1]/div/div/div/div[2]/p[2]/a').click()
            for i in range(20):
                if os.path.exists('c:\\Users\\leowy\\Downloads\\historyIndex.xls'):
                    break
                time.sleep(1)
            os.rename('c:\\Users\\leowy\\Downloads\\historyIndex.xls', f'c:\\Users\\leowy\\Downloads\\MSCI {index} USD {index_level} Monthly.xls')
            frequency_field = driver.find_element('id', 'updateFrequencyRegion')
            frequency_field_select = Select(frequency_field)
            frequency_field_select.select_by_value('DAILY')
        for start_date, end_date in zip(pd.date_range('2001-01-01', 'today', freq='5BAS-JAN').strftime('%b %d, %Y'),
                                        pd.date_range('2005-01-01', 'today', freq='5BA-DEC').union(pd.date_range(pd.to_datetime('today') + pd.tseries.offsets.BMonthEnd(-1, 'D'), periods=1, freq=pd.tseries.offsets.BMonthEnd(-1, 'D'))).strftime('%b %d, %Y')):
            if os.path.exists(f'c:\\Users\\leowy\\Downloads\\MSCI {index} USD {index_level} Daily {start_date[-4:]}-{end_date[-4:]}.xls'):
                continue
            start_date_field = driver.find_element('id', 'startDateFilterShow')
            start_date_field.clear()
            start_date_field.send_keys(start_date)
            driver.find_element('xpath', '/html/body').click()
            end_date_field = driver.find_element('id', 'endDateFilterShow')
            end_date_field.clear()
            end_date_field.send_keys(end_date)
            driver.find_element('xpath', '/html/body').click()
            driver.find_element('xpath', '/html/body/div[1]/div/section/div/div[4]/div[4]/div[2]/form/div/div/div[8]/div/div/button').click()
            WebDriverWait(driver, 60).until(EC.invisibility_of_element(('id', 'loading-image-graph')))
            time.sleep(1)
            driver.find_element('xpath', '/html/body/div[1]/div/div/div/div[2]/p[2]/a').click()
            for i in range(20):
                if os.path.exists('c:\\Users\\leowy\\Downloads\\historyIndex.xls'):
                    break
                time.sleep(1)
            os.rename('c:\\Users\\leowy\\Downloads\\historyIndex.xls', f'c:\\Users\\leowy\\Downloads\\MSCI {index} USD {index_level} Daily {start_date[-4:]}-{end_date[-4:]}.xls')

In [12]:
driver.quit()